Hi


In [12]:
#########################
# read all files from get_familiar
# save them in a dictionary of datasets

import pandas as pd
import os

data_dir = '../d1_get_familiar/data/get_familiar'
datasets = {}
for file in os.listdir(data_dir):
    if file.endswith('.feather'):
        dataset_name = file[:file.index('_person')]
        datasets[dataset_name] = pd.read_feather(os.path.join(data_dir, file))

In [13]:
# the names of the datasets
print(datasets.keys(), '\n')
# check the sizes of datasets
total_size = 0
for name, df in datasets.items():
    size = df.memory_usage(deep=True).sum()
    total_size += size
    print(f"Dataset '{name}' size: {size / (1024 ** 2):.2f} MB")
print(f"\nTotal size of all datasets: {total_size / (1024 ** 2):.2f} MB")

dict_keys(['breathing', 'ecg', 'insulin', 'glucose', 'summary', 'accel']) 

Dataset 'breathing' size: 9.91 MB
Dataset 'ecg' size: 99.12 MB
Dataset 'insulin' size: 0.00 MB
Dataset 'glucose' size: 0.09 MB
Dataset 'summary' size: 6.94 MB
Dataset 'accel' size: 79.30 MB

Total size of all datasets: 195.36 MB


In [15]:
print(datasets['glucose'].info())
datasets['insulin']

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1438 entries, 2014-10-01 19:14:00 to 2014-10-06 16:54:02
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   glucose  1438 non-null   float64
 1   type     1438 non-null   object 
dtypes: float64(1), object(1)
memory usage: 33.7+ KB
None


,fast_insulin,slow_insulin
datetime,,
2014-10-01 10:06:00,7,NaN
2014-10-01 16:50:00,4,NaN
2014-10-01 19:28:00,6,NaN
2014-10-01 22:27:00,8,NaN
2014-10-01 23:48:00,0,31.0
2014-10-02 00:45:00,2,NaN
2014-10-02 10:10:00,3,NaN
2014-10-02 12:34:00,12,NaN
2014-10-02 21:24:00,5,NaN


# Data loading from Sasha's preprocessing

Residual Network ResNet are a improvement on CNN, they adress the vanishing gradient issue so they outperform CNNs in complex task (e.g. classification) especially for deep networks
(https://www.analyticsvidhya.com/blog/2021/08/how-to-code-your-resnet-from-scratch-in-tensorflow/)

The goal is at first to use a basic CNN 1D with functionnal API to allow more flexibility when complexifing the model. C1D is good for our purpose because it is good at recognozing pattern in a short period interval -> What we want here to predict a dip in glucose level based on our ECG (c.f. Sasha's article on link between ECG and Glucose level)


Do the same with an LSTM which is more suitable for long term recognision

# C1D Architecture

The idea is to create a basic architecture for a CNN where we would have access to the output of each layer to be able to combine with another model (LSTM)  

In [ ]:
# Create a very simple CNN model for testing using functional API
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def create_simple_cnn(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv1D(32, kernel_size=3, activation='relu')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(64, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(1, activation='linear')(x)

    model = keras.Model(inputs, outputs)
    return model


/home/danielfarkas/.pyenv/versions/3.12.9/envs/hypopredict/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-12-09 15:16:40.820348: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 98, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 49, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 47, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 23, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1472)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       188,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 195,009 (761.75 KB)

 Trainable params: 195,009 (761.75 KB)

 Non-trainable params: 0 (0.00 B)

# LSTM Architecture

Same idea as above

In [ ]:
#Create a very simple LSTM model for testing using functional API
def create_simple_lstm(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.LSTM(50, return_sequences=True)(inputs)
    x = layers.LSTM(50)(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(1, activation='linear')(x)

    model = keras.Model(inputs, outputs)
    return model
